https://www.philipzucker.com/ground_kbo/



What are our examples

Example mul succ plus


functional programs
plus(z,X) = X
plus(succ(x), y) = succ(plus(x, y))
mul(succ(x), y) = plus(mul(x, y), y)

ackermann
ack(0, y) -> succ(y)
ack(succ(x), 0) -> ack(x, succ(0))
ack(succ(x), succ(y)) -> ack(x, ack(succ(x), y))

note that x appears twice

macro expansion
foo(x,y,z) = bar(x,x,y,z,z,z,z)


association
(x + y) + z -> x + (y + z)

distribution
x * (y + z) -> x * y + x * z

negation normal form




Homeomorphic embedding dignifies structural shrinking is well founded.

Subterms are smaller
But accoutning for variables.

A term is bigger than another if you can delete nodes to get it.

This is the "closure" of being a subterm in combination with treating variables correctly.


In [ ]:
def decls(t):
    

In [50]:
log = False
def log_arguments(func):
    def wrapper(*args, **kwargs):
        # Print the function name and arguments
        global log
        if log:
            print(f"Calling {func.__name__} with arguments: {args} and keyword arguments: {kwargs}")
        res = func(*args, **kwargs)  # Call the original function
        if log:
            print(f"Calling {func.__name__} with arguments: {args} and keyword arguments: {kwargs} returned {res}")
        return res
    return wrapper

In [19]:
from z3 import *

# definition traat 5.4.2
@log_arguments
def homeo(s,t):
    if is_var(s) and s.eq(t):
        return True
    elif is_app(s) and is_app(t) and s.decl() == t.decl():

        return all(homeo(s.arg(i),t.arg(i)) for i in range(s.num_args()))
    elif is_app(s) and any(homeo(s.arg(i), t) for i in range(s.num_args())):
        return True
    else:
        return False

v0,v1,v2 = Var(0, IntSort()), Var(1, IntSort()), Var(2, IntSort())
assert homeo(Int('x'), Int('x'))
assert not homeo(Int('x'), Int('y'))
assert homeo(Int('x')+Int('y'), Int('x')+Int('y'))
assert not homeo(Int('x')+Int('y'), Int('y')+Int('x'))
assert homeo(v0,v0)
assert not homeo(v0,v1)
assert homeo(v0 + 1, v0)
assert not homeo(v0, v0 + 1)

Calling homeo with arguments: (x, x) and keyword arguments: {}
Calling homeo with arguments: (x, y) and keyword arguments: {}
Calling homeo with arguments: (x + y, x + y) and keyword arguments: {}
Calling homeo with arguments: (x, x) and keyword arguments: {}
Calling homeo with arguments: (y, y) and keyword arguments: {}
Calling homeo with arguments: (x + y, y + x) and keyword arguments: {}
Calling homeo with arguments: (x, y) and keyword arguments: {}
Calling homeo with arguments: (Var(0), Var(0)) and keyword arguments: {}
Calling homeo with arguments: (Var(0), Var(1)) and keyword arguments: {}
Calling homeo with arguments: (Var(0) + 1, Var(0)) and keyword arguments: {}
Calling homeo with arguments: (Var(0), Var(0)) and keyword arguments: {}
Calling homeo with arguments: (Var(0), Var(0) + 1) and keyword arguments: {}


In [56]:
def mem(t,s):
    if t.eq(s):
        return True
    elif is_app(s):
        return any(mem(t,si) for si in s.children())
    else:
        return False

@log_arguments
def lpo(s,t):
    if s.eq(t): # fast track for equality
        return False
    elif is_var(t):
        return mem(t,s) #lpo1
    elif is_var(s):
        return False
    elif is_app(s):
        f, g = s.decl(), t.decl()
        if any(si.eq(t) or lpo(si,t) for si in s.children()): #lpo2a
            return True
        if all(lpo(s,ti) for ti in t.children()):
            if f == g: #lpo2c
                for si,ti in zip(s.children(),t.children()):
                    if si.eq(ti):
                        continue
                    elif lpo(si,ti):
                        return True
                    else:
                        return False
                raise Exception("Should be unreachable if s == t", s,t)
            elif (f.name(), f.arity()) > (g.name(), g.arity()): # lpo2b
                return True
            else:
                return False
        else:
            return False
    else:
        raise Exception("Unexpected case lpo", s,t)
x,y,z,e = Ints('x y z e')
f = Function("f", IntSort(), IntSort(), IntSort())
i = Function("i", IntSort(), IntSort())
log = False
assert not lpo(Int('x'), Int('y'))
assert lpo(Int("y"), Int("x"))
assert not lpo(v0, x)
assert not lpo(x, v0)
assert lpo(x + y, x)
assert lpo(f(v0, e), v0)
assert lpo(i(e), e)
assert lpo(i(f(v0,v1)), f(i(v1),i(v0)))
assert lpo(f(f(v0, v1), v2), f(v0, f(v1, v2)))


# anti symmettry
assert not lpo(x, x + y)
assert not lpo(v0, f(v0, e))
assert not lpo(e, i(e))
assert not lpo(f(i(v1),i(v0)), i(f(v0,v1)))
assert not lpo(f(v0, f(v1, v2)), f(f(v0, v1), v2))
assert not lpo(f(v0, f(v1, v2)), f(f(v0, v1), v2))


We have an intuition that lexicographic comparison of trees is reasonable.

It isn't a wll founded order though

0 + (1 + 0) ->  (0 + (0 + 1)) ->  


lpo.
Brute force a simplification order,
 but once we know that, use naive lex



def lpo

https://www.cs.tau.ac.il/~nachum/papers/printemp-print.pdf 33 examples of termination

`f(f(x)) -> g(f(x))`

```
f(f(x)) -> g(f(x))
g(g(x)) -> f(x)
```
lex combo of orderings works


`f(f(x)) -> f(g(f(x)))` adjencet fs decrease







https://en.wikipedia.org/wiki/Path_ordering_(term_rewriting)

https://academic.oup.com/comjnl/article/34/1/2/427931 An Introduction to Knuth–Bendix Completion Ajj dick

https://link.springer.com/article/10.1007/s10817-006-9031-4 Things to Know when Implementing KBO

https://www.cs.unm.edu/~mccune/mace4/manual/2009-02A/term-order.html

Sims had anb expalantion in terms of wreath product for rpo


https://hol-light.github.io/tutorial.pdf

"The most basic way the LPO orders terms is by use of the weighting function on
the top-level function symbols. For example, if we give multiplication a higher weight
than addition (by putting it later in the list lis above), this will make the distributive
law (x+y)· z = x· z +y · z applicable left-to-right, since it brings a ‘simpler’ operator
to the top, even though it actually increases the actual size of the term. In cases where
the top operators are the same, the LPO assigns preferences lexicographically left-toright, so that the ordering of the leftmost arguments are considered first, and only in
the event of a tie are the others considered one-by-one. This, for example, will tend to
drive associative laws like (x + y) + z = x + (y + z) in the left-to-right direction since
x is simpler than x + y. We use a generic function lexord to apply a basic ordering
ord lexicographically, returning false if the lists are of different lengths:"

